# 1. Setup

In [1]:
from mobilkit.umni import *

In [2]:
from scipy.spatial import cKDTree

# 2. Prepare data

## 2.1. Filter eligible OD pairs

In [3]:
max_speeds = {k: v * U.MPH2MPS for k, v in D(
    BIKE = 16, DRIVE = 70, TRANSIT = 20, WALK = 3.1
).items()}

In [5]:
max_time = 3600 # 1 hour of travel time

In [6]:
def get_eligible_odps(rgn, year, speeds=max_speeds, 
                      max_time=max_time, overwrite=False):
    """Create candidate OD pairs for zone centroids for each mode by 
    considering pairs within a maximum distance reachable by each 
    mode within a given maximum travel time.

    Args:
        rgn (str): Label of the study region (to get zones table).
        year (int): Year of the zones dataset.
        speeds (dict): Maximum speed (m/s) of each travel mode.
        max_time (float): Maximum duration (s) of travel,
            used to compute the radius of reach of each mode.
    Returns:
        (Pdf): Table of origin (src) and destination (trg) geoid
            and coordinates, along with mode and scale.
    """
    outpath = Path(f'../data/od/odp/{rgn.lower()}_{year}.parquet')
    if outpath.exists() and not overwrite:
        return pd.read_parquet(outpath)
    zones = (gpd.read_parquet(f'../data/zones/{rgn.lower()}_{year}.parquet')
             .to_crs(CRS_M).assign(geometry=lambda df: df.centroid)
             [['geoid', 'scale', 'geometry']].reset_index(drop=1))
    xy = mk.geo.gdf2pdf(zones, 'x', 'y', CRS_M)
    latlon = mk.geo.gdf2pdf(zones, LON, LAT, CRS_DEG)
    zones = (pd.concat([zones, xy, latlon], axis=1)
             .drop(columns='geometry').astype(D(x=np.int32, y=np.int32)))
    res = []
    pbar = tqdm(total=zones['scale'].nunique() * len(speeds))
    for scale, df in zones.groupby('scale', sort=False):
        tree = cKDTree(df[['x', 'y']])
        for mode, speed in speeds.items():
            pbar.update()
            pbar.set_description(f'{scale} -> {mode}')
            odp = tree.query_pairs(max_time * speed, output_type='ndarray')
            odp = pd.concat([
                df.iloc[odp[:, i]][['geoid', 'lon', 'lat']]
                .rename(columns=(kind + '_{}').format).reset_index(drop=1)
                for i, kind in enumerate(['src', 'trg'])
            ], axis=1).assign(scale=scale, mode=mode)
            res.append(odp)
    odp = pd.concat(res).reset_index(drop=True).astype({
        x: np.float32 for x in ['src_lon', 'src_lat', 'trg_lon', 'trg_lat']
    }).astype(CAT)
    odp.to_parquet(U.mkfile(outpath))
    return odp

odpIN10 = get_eligible_odps('IN', 2010, overwrite=0).disp() # t=0:04 (prev. t=3:49)
# odpIN20 = get_eligible_odps('IN', 2020, overwrite=0).disp(0) # t=0:05
odpMSA10 = get_eligible_odps('MSA', 2010, overwrite=0).disp(0) # t=0:45
# odpMSA20 = get_eligible_odps('MSA', 2020, overwrite=0).disp(0) # t=0:55
# x = get_eligible_odps('IN', 2010, overwrite=0); x

3,810,346 rows x 8 cols; Memory: 81.3 MiB


,src_geoid,src_lon,src_lat,trg_geoid,trg_lon,trg_lat,scale,mode
,<category>,<float32>,<float32>,<category>,<float32>,<float32>,<category>,<category>
0,18115,-84.965012,38.950115,18155,-85.036972,38.826218,COUNTY,BIKE


47,324,439 rows x 8 cols; Memory: 1180.3 MiB


,src_geoid,src_lon,src_lat,trg_geoid,trg_lon,trg_lat,scale,mode
,<category>,<float32>,<float32>,<category>,<float32>,<float32>,<category>,<category>


In [7]:
pd.concat([v['scale'].value_counts().rename(k) for k, v in [
    ('IN', odpIN10), ('MSA', odpMSA10)]], axis=1).fillna(0).astype(int)

,IN,MSA
scale,,
BG,3449964,0
TRACT,359603,47320808
COUNTY,779,3631


In [8]:
pd.concat([v['mode'].value_counts().rename(k) for k, v in [
    ('IN', odpIN10), ('MSA', odpMSA10)]], axis=1)

,IN,MSA
mode,,
DRIVE,2674787,29468465
TRANSIT,611622,9975978
BIKE,478685,7397408
WALK,45252,482588


## Download from Google
The [old disastrous] code is written in `./gdm.py`.

## Download with OSRM
- The [old] code that uses the OSRM web server is written in `$MK/spr_4711/code/_old/osrm_web.ipynb`

## Using OD pairs

In [19]:
# def get_osrm_table(rgn, year, scale=None, mode='DRIVE', overwrite=False):
#     outpath = Path(f'../data/travel_time/time_{rgn.lower()}_{year}.parquet')
#     if outpath.exists() and not overwrite:
#         return pd.read_parquet(outpath)
#     od_all = (get_eligible_odps(rgn, year).query('mode=="{}"{}'.format(
#                   mode, f' & scale=="{scale}"' if scale else ''))
#                   .reset_index(drop=1))
#     ##
#     od_all = od_all.head(10)
#     ##
#     idx = np.concatenate([np.arange(0, len(od_all), 100), [len(od_all)]])
#     cols = ['geoid', 'lon', 'lat']
#     res = []
#     for i, j in tqdm(zip(idx[:-1], idx[1:]), total=len(idx) - 1):
#         od = od_all.iloc[i: j]
#         # waypoints (unique nodes in the OD table)
#         wp = pd.concat([od.filter(like=direc).rename(
#             columns=lambda x: x.replace(direc + '_', ''))
#             for direc in ['src', 'trg']]
#         ).drop_duplicates().sort_values('geoid')
#         wp = wp.reset_index(drop=1).rename_axis('i').reset_index()
#         od = od.merge(wp[['i', 'geoid']].rename(columns='src_{}'.format))
#         od = od.merge(wp[['i', 'geoid']].rename(columns='trg_{}'.format))
#         xy = ';'.join([f'{r.lon},{r.lat}' for _, r in wp.iterrows()])
#         src = ';'.join(od['src_i'].astype(str).unique())
#         trg = ';'.join(od['trg_i'].astype(str).unique())
#         print(src)
#         print(trg)
#         print(wp)
#         print(od[['src_geoid','trg_geoid','src_i','trg_i']])
#         # return od
#         base = 'http://router.project-osrm.org/table/v1/driving'
#         url = f'{base}/{xy}?sources={src}&destinations={trg}'
#         r = requests.get(url)
#         df = Pdf(r.json()['durations'])
#         return df
#         # df = Pdf(r.json()['durations'], index=od.src_geoid, columns=od.trg_geoid)
#         df = (df.reset_index().melt('src_geoid')
#               .query('value > 0').drop_duplicates().reset_index(drop=1))
#         res.append(df)
#     res = (pd.concat(res).rename(columns=D(value='time'))
#            .astype(D(src_geoid=CAT, trg_geoid=CAT)).reset_index(drop=1))
#     res.to_parquet(U.mkfile(outpath))
#     return res

# x = get_osrm_table('IN', 2010, 'COUNTY', overwrite=1); x

In [20]:
# def get_clust(od, mode=None):
#     if isinstance(mode, str):
#         od = od[od['mode'] == mode]
#     od = Pdf(D(o=od.src_geoid.cat.codes, d=od.trg_geoid.cat.codes))
#     g = ig.Graph.TupleList(zip(*od.values.T), directed=False)
#     clust = (Seq(dict(enumerate(g.community_label_propagation())), name='node')
#              .explode().rename_axis('clust').reset_index().astype(np.int32))
#     return clust

# cIN = get_clust(odpIN10, 'DRIVE').disp() # t=0:04
# # cMSA = get_clust(odpMSA10).disp() # t=0:47

In [15]:
# from time import time

# import networkx as nx
# from scipy.cluster.hierarchy import dendrogram
# from sklearn.cluster import KMeans, DBSCAN

In [16]:
# od = odpIN10.query('scale=="TRACT" & mode=="DRIVE"')
# i2v = dict(enumerate(sorted(set(od.src_geoid) | set(od.trg_geoid))))
# v2i = {x: i for i, x in i2v.items()}
# od = (Pdf(D(o=od.src_geoid.map(v2i), d=od.trg_geoid.map(v2i)))
#       .astype(np.int32).drop_duplicates())
# g = ig.Graph.TupleList(zip(*od.values.T), directed=False)
# print(f'nodes={g.vcount()}, edges={g.ecount()}')
# # # print(g.community_edge_betweenness())
# # print(g.community_fastgreedy())

nodes=1511, edges=245866


In [17]:
# gNx = nx.from_pandas_edgelist(od, 'o', 'd')
# print(len(gNx), len(gNx.edges()))
# cover = list(nx.algorithms.approximation.min_weighted_vertex_cover(gNx))
# print(len(cover))

1511 245866
1510


## Using zones

In [21]:
def get_durations(od, geoids, mode='driving'):
    """Obtain the full OD duration matrix for the zones included 
    in the given `od` table using the OSRM API.

    Parameters
    ----------
    od : pd.DataFrame
        Must have the columns {'src', 'trg'} x {'geoid', 'lon', 'lat'}
    
    Returns
    -------
    pd.DataFrame
        ...
    """
    od = od.merge(geoids.rename('src_geoid'), on='src_geoid')
    od = od.merge(geoids.rename('trg_geoid'), on='trg_geoid')
    wp = pd.concat([od.filter(like=direc).rename(
        columns=lambda x: x.replace(direc + '_', '')) 
        for direc in ['src', 'trg']
    ]).drop_duplicates(subset='geoid').reset_index(drop=1)
    wp = wp.rename_axis('i').reset_index()
    od = od.merge(wp[['i', 'geoid']].rename(columns='src_{}'.format))
    od = od.merge(wp[['i', 'geoid']].rename(columns='trg_{}'.format))
    xy = ';'.join([f'{r.lon},{r.lat}' for _, r in wp.iterrows()])
    base = f'http://router.project-osrm.org/table/v1/{mode}'
    url = f'{base}/{xy}'
    r = requests.get(url)
    df = Pdf(r.json()['durations'],
             index=wp.geoid.rename('src_geoid'),
             columns=wp.geoid.rename('trg_geoid'))
    df = df.reset_index().melt('src_geoid', value_name='duration')
    df = df.merge(od[['src_geoid', 'trg_geoid']])
    return df

In [22]:
# def get_clusters(rgn, year, scale, mode, modes=modes, tmax=3600, nmax=100):
#     fname = f'{rgn.lower()}_{year}.parquet'
#     zones = (gpd.read_parquet(f'../data/zones/{fname}')
#              .query(f'scale == "{scale}"').reset_index(drop=1)
#              .assign(geometry=lambda df: df.to_crs(CRS_M).centroid))
#     xy = mk.geo.gdf2pdf(zones, 'x', 'y', crs=CRS_M)
#     latlon = mk.geo.gdf2pdf(zones, 'lon', 'lat', crs=CRS_DEG)
#     zones = pd.concat([zones['geoid'], latlon, xy], axis=1)
#     odp = (pd.read_parquet(f'../data/travel_time/odps_{fname}')
#            .query(f'scale == "{scale}" & mode == "{mode}"')
#            .drop(columns=['scale', 'mode']).reset_index(drop=1))
#     dmax = tmax * modes.loc[mode].max_speed
#     model = AgglomerativeClustering(n_clusters=None, distance_threshold=dmax)
#     zones['clust'] = model.fit(xy).labels_
#     durations = []
#     for clust, df in tqdm(zones.groupby('clust')):
#         dur = get_durations(odp, df['geoid'].head(nmax))
#         durations.append(dur.assign(clust=clust))
#         idx = odp.merge(dur, on=('src_geoid', 'trg_geoid')).index
#         if clust == 1:
#             return dur
#             return odp.merge(dur)
#         odp.drop(idx, inplace=True)
#         return odp
#     dur = pd.concat(durations).reset_index()
#     return dur

# x = get_clusters('IN', 2010, 'COUNTY', 'DRIVE'); x

In [23]:
def get_full_matrix(base_url, nodes):
    url = '{}/{}'.format(base_url, ';'.join([
        f'{x},{y}' for x, y in zip(nodes['lon'], nodes['lat'])]))
    dur = Pdf(requests.get(url).json()['durations'],
              index=nodes['geoid'].rename('src_geoid'),
              columns=nodes['geoid'].rename('trg_geoid'))
    return dur.reset_index().melt('src_geoid', value_name='duration')

In [ ]:
def get_clusters(rgn, year, scale, mode, modes=modes, batch_size=100, 
                 tmax=3600, overwrite=False):
    outpath = Path('../data/travel_time/osrm/{}_{}_{}_{}.parquet'.format(
        rgn.lower(), year, scale.lower(), mode.lower()))
    if outpath.exists() and not overwrite:
        return pd.read_parquet(outpath)
    # edgelist (OD pairs)
    E = pd.read_parquet(f'../data/travel_time/odps_{rgn.lower()}_{year}.parquet')
    E = E[(E['scale'] == scale) & (E['mode'] == mode)].reset_index(drop=1)
    E = E.drop(columns=['scale', 'mode'])
    # add edges in the reverse direction
    src = E.filter(like='src').rename(columns=lambda x: x.replace('src_', ''))
    trg = E.filter(like='trg').rename(columns=lambda x: x.replace('trg_', ''))
    E_flip = pd.concat([src.rename(columns='trg_{}'.format), 
                        trg.rename(columns='src_{}'.format)], axis=1)
    E = pd.concat([E, E_flip]).reset_index(drop=1)#.rename_axis('eid')
    print(E.shape)
    # unique nodes
    V = pd.concat([src, trg]).drop_duplicates(subset='geoid')
    V = V.sort_values('geoid').reset_index(drop=1)
    V_gdf = mk.geo.pdf2gdf(V, crs=CRS_DEG).to_crs(CRS_M)
    V = pd.concat([V, mk.geo.gdf2pdf(V_gdf, 'x', 'y', CRS_M)], axis=1)
    V = V.rename_axis('vid')
    # create clusters based on max. distance reachable by the mode
    dmax = tmax * modes.loc[mode].max_speed
    # model = AgglomerativeClustering(n_clusters=None, distance_threshold=dmax)
    min_samp = D(COUNTY=2, TRACT=2, BG=50)[scale]
    model = DBSCAN(eps=dmax, min_samples=min_samp)
    # nmax, step = D(COUNTY=(100, 10), TRACT=(1500, 100), BG=(10000, 1000))[scale]
    # for nclust in tqdm(np.arange(0, nmax, step)[1:]):
        # model = AgglomerativeClustering(n_clusters=nclust)
    # model = KMeans(nclust)
    V['clust'] = model.fit(V[['x', 'y']]).labels_
    durations = []
    mode_lab = D(DRIVE='driving', BIKE='bike', WALK='walk')[mode]
    base_url = f'http://router.project-osrm.org/table/v1/{mode_lab}'
    for clust, V2 in V.groupby('clust'):                                ## for each cluster
        idx = list(np.arange(0, len(V2) + 1, batch_size)) + [len(V2)]
        for i, j in zip(idx[:-1], idx[1:]):                             ## for each batch
            # iters += 1
            V3 = V2.iloc[i: j].reset_index(drop=1)
            try:
                url = base_url + '/' + ';'.join([
                    f'{x},{y}' for x, y in zip(V3.lon, V3.lat)])
                # dur = requests.get(url).json()['durations']
                dur = Pdf(1, index=V3.geoid.rename('src_geoid'),
                            columns=V3.geoid.rename('trg_geoid'))
                dur = dur.reset_index().melt('src_geoid', value_name='duration')
                E2 = E.merge(dur, 'left').dropna(subset='duration')
                durations.append(E2.assign(clust=clust))
                E.drop(E2.index, inplace=True, errors='ignore')
                # Efinal = E.drop(E2.index, errors='ignore')
            except Exception as e:
                print(f'ERROR={e}, region={rgn}, year={year}, scale={scale}',
                    f'mode={mode}, cluster={clust}, nnodes={len(V2)}, batch={i}')
    print(f'leftover edges = {len(E)}')
    # print(f'clusters={nclust}, iterations={iters}, remaining_edges={len(Efinal)}')
    # dur = pd.concat(durations).reset_index(drop=1)
    # dur.to_parquet(U.mkfile(outpath))
    # return dur

# %time print('IN county'); get_clusters('IN', 2010, 'COUNTY', 'DRIVE') # t=2:13
# %time print('IN tract'); get_clusters('IN', 2010, 'TRACT', 'DRIVE') # 6:50
# %time print('IN bg'); get_clusters('IN', 2010, 'BG', 'DRIVE') # t=12:54
# %time print('MSA county'); get_clusters('MSA', 2010, 'COUNTY', 'DRIVE') # t=11:05
# %time print('MSA tract'); get_clusters('MSA', 2010, 'TRACT', 'DRIVE')
# x = get_clusters('IN', 2010, 'COUNTY', 'DRIVE', overwrite=1); x
x = get_clusters('IN', 2010, 'TRACT', 'DRIVE', overwrite=1); x

In [ ]:
odpIN10.query('mode=="DRIVE"')['scale'].value_counts()

In [ ]:
odpIN10['mode'].value_counts()

In [ ]:
odpMSA10['mode'].value_counts()

# Archive

In [ ]:
# def get_eligible_odps_one_region(big_df, max_duration=max_duration, 
#                                  max_speeds=max_speeds, workers=4):
#     res = []
#     for _, df2 in big_df.groupby('scale', sort=False):
#         if len(df2) <= 1: continue
#         tree = cKDTree(df2[['x', 'y']])
#         df = []
#         for mode, max_speed in max_speeds.items(): ## mode (bike/drive...)
#             radius = max_speed * max_duration
#             odp = tree.query_ball_point(df2[['x', 'y']], radius, workers=workers)
#             odp = sum([[(i, x) for x in p] for i, p in enumerate(odp)], [])
#             odp = Pdf(odp, columns=['src_id', 'trg_id']).assign(mode=mode)
#             df.append(odp)
#         df = pd.concat(df).reset_index(drop=True)
#         res.append(pd.concat([df[['mode']]] + [
#             df2.iloc[df.pop(f'{kind}_id')][['geoid', LON, LAT]]
#             .rename(columns=(kind + '_{}').format).reset_index(drop=True)
#             for kind in ['src', 'trg']
#         ], axis=1))
#     return pd.concat(res).reset_index(drop=True)

In [ ]:
# def get_eligible_odps(zones, imp_states=[], imp_cbsas=[],
#                       njobs=24, overwrite=False):
#     """
#     Filter zone OD pairs for the 3 spatial scales by mode within either 
#     the given states or the given CBSAs by limiting them within the 
#     circle induced by the maximum modal distance coverable within the 
#     given maximum duration.
#     - zones: Gdf
#         Zones table for all the US and all scales
#     - imp_states: list[str]
#         List of US states to be included (full names only)
#     - imp_cbsas: list[str]
#         List of the FIPS codes of the CBSAs to be included
#     - max_duration: int
#         Maximum duration of travel in seconds
#     - max_speeds: dict
#         Maximum speed for each mode in m/s
#     - workers: int
#         Number of workers for parallel spatial tree search
#     """
#     outpath = Path(f'../data/deterrence/od_pairs.parquet')
#     if outpath.exists() and not overwrite:
#         return pd.read_parquet(outpath)
#     zones = (zones.assign(geometry=zones.to_crs(CRS_M).centroid)
#              .assign(x=lambda df: df.geometry.x, y=lambda df: df.geometry.y)
#              .to_crs(CRS_DEG)
#              .assign(lon=lambda df: df.geometry.x, lat=lambda df: df.geometry.y)
#              .astype({'lon': np.float32, 'lat': np.float32})
#              [['geoid', 'scale', 'state', 'cbsa', 'x', 'y', 'lon', 'lat']])
#     dfs = []
#     for col, rgns in [('cbsa', imp_cbsas), ('state', imp_states)]:
#         for _, df in zones[zones[col].isin(rgns)].groupby(col):
#             if len(df) <= 1: continue
#             dfs.append(df[['geoid', 'scale', 'x', 'y', 'lon', 'lat']])
#     odp = pqdm(dfs, get_eligible_odps_one_region, n_jobs=njobs)
#     odp = (pd.concat(odp).query('src_geoid != trg_geoid')
#            .drop_duplicates(subset=['src_geoid', 'trg_geoid', 'mode'])
#            .dropna().reset_index(drop=True).astype(CAT))
#     odp.to_parquet(U.mkfile(outpath), compression='gzip')
#     return odp

# od10 = get_eligible_odps(zones10, ['Indiana'], list(imp_cbsas.cbsa), 
#                          overwrite=0).disp() # t=9:46:30

In [ ]:
# x = (od10.assign(scale=od10.src_geoid.str.len()
#                  .map({5: 'county', 11: 'tract', 12: 'bg'}))
#      .groupby(['scale', 'mode']).size().rename('n_odp').reset_index()); x

In [ ]:
# x.groupby('scale')['n_odp'].sum()

In [ ]:
zones10['scale'].value_counts()